<a href="https://colab.research.google.com/github/MHusni1604/Pengantar-Anlisis-Data-Panel/blob/main/Tugas_Minggu_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis Dampak Trump Tariffs Terhadap Harga Saham LQ45
---
![Trump Tariffs](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Trump_showing_a_chart_with_reciprocal_tariffs_%28cropped%29.jpg/500px-Trump_showing_a_chart_with_reciprocal_tariffs_%28cropped%29.jpg)

## Pendahuluan

Pada awal tahun 2025, kebijakan tarif impor baru yang dikenal sebagai **"Trump Tariffs"** diumumkan, menaikkan tarif impor barang ke Amerika Serikat secara signifikan dari sekitar 2,5% menjadi 27% [\[1\]](https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration). Kebijakan ini menimbulkan ketidakpastian di pasar global, termasuk berdampak pada pergerakan harga saham di negara berkembang seperti Indonesia.

Indeks **LQ45** di Bursa Efek Indonesia mencerminkan performa 45 saham paling likuid dan menjadi indikator utama respon pasar domestik terhadap perubahan ekonomi global serta indikator makroekonomi seperti suku bunga acuan Bank Indonesia (BI Rate), inflasi, dan nilai tukar Rupiah terhadap Dolar AS [\[2\]](https://id.wikipedia.org/wiki/Indeks_LQ45).

Penelitian ini bertujuan untuk menganalisis bagaimana kebijakan **Trump Tariffs** dan variabel makroekonomi memengaruhi kategori perubahan harga saham harian perusahaan dalam indeks LQ45. Variabel target dikategorikan berdasarkan perubahan harga (misalnya naik, turun, atau stabil), sehingga pendekatan yang digunakan adalah **model multinomial logit**. Model ini memungkinkan estimasi probabilitas suatu saham berada dalam salah satu dari beberapa kategori hasil (multikategori), berdasarkan kondisi ekonomi dan kebijakan yang berlaku.

Dengan menggunakan pendekatan data panel, analisis ini mempertimbangkan dimensi waktu dan entitas (perusahaan), serta heterogenitas antar saham. Model ini diharapkan dapat memberikan pemahaman yang lebih komprehensif mengenai dinamika pasar saham Indonesia sebagai respon terhadap tekanan eksternal dan indikator ekonomi domestik secara kuantitatif.


## Metodologi

### Sumber Data  
Penelitian ini menggunakan data sekunder yang diperoleh dari dua sumber utama: [Yahoo Finance](https://finance.yahoo.com) untuk data harga penutupan saham harian dan nilai tukar Rupiah terhadap Dolar AS, serta [situs resmi Bank Indonesia](https://www.bi.go.id) untuk data suku bunga acuan (BI Rate) dan tingkat inflasi. Sampel penelitian terdiri dari 45 perusahaan yang tergabung dalam Indeks LQ45 di Bursa Efek Indonesia dengan total observasi sebanyak 4050 data harian.

### Spesifikasi Model  
Analisis dilakukan menggunakan **Multinomial Logit Model**, sebuah pendekatan regresi untuk variabel dependen kategorik dengan lebih dari dua kelas. Dalam konteks ini, variabel target merupakan hasil kategorisasi dari perubahan harga saham harian menjadi tiga kelas, misalnya:
- `0`: Harga turun  
- `1`: Harga stabil  
- `2`: Harga naik  

Model ini mengestimasi probabilitas saham termasuk ke dalam salah satu dari ketiga kategori tersebut, berdasarkan faktor-faktor ekonomi makro dan kebijakan tarif impor.

Model dapat dituliskan secara umum sebagai berikut:

$$
P(Y_{it} = k) = \frac{e^{X_{it} \beta_k}}{\sum_{j=0}^2 e^{X_{it} \beta_j}}, \quad \text{untuk } k = 1, 2
$$

Dengan:
- $Y_{it}$: Kategori perubahan harga saham perusahaan $i$ pada hari ke-$t$  
- $X_{it}$: Vektor variabel independen seperti `Trump Tariffs`, `BI-Rate`, `Inflasi`, `USD/IDR`, dan lag harga saham (`Close`)  
- $\beta_k$: Koefisien regresi untuk kategori $k$ (dengan referensi kategori dasar, misalnya `0`)  

### Alasan Pemilihan Metode  
Multinomial logit dipilih karena variabel target bersifat kategorik dan tidak berurutan, sehingga pendekatan regresi linier tidak lagi tepat. Model ini memungkinkan analisis probabilitas relatif dari perubahan harga saham dalam tiga kategori yang berbeda, tergantung pada kondisi makroekonomi dan kebijakan eksternal.  

Selain itu, penggunaan **data panel** dengan indeks waktu dan perusahaan memungkinkan pemodelan dinamika harga saham yang memperhitungkan lag serta perbedaan antar entitas. Dengan menambahkan lag dari variabel prediktor seperti harga penutupan (`Close_lag1`) dan indikator ekonomi lainnya, model juga mengakomodasi efek waktu yang bersifat temporal pada keputusan pasar.

Model ini memberikan kerangka statistik yang sesuai untuk mengevaluasi dampak kebijakan **Trump Tariffs** terhadap arah perubahan harga saham perusahaan di indeks LQ45 secara lebih komprehensif.

# Hasil dan Pembahasan

## Import Library

In [2]:
pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 988.0 kB/s eta 0:00:00


In [3]:
# ==========================================
# 📦 General Purpose Libraries
# ==========================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================================
# 📈 Financial Data
# ==========================================
import yfinance as yf

# ==========================================
# 📊 Statistical & Econometrics Tools
# ==========================================
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

# ==========================================
# 📐 Statistical Tests
# ==========================================
from scipy.stats import shapiro

# ==========================================
# 🤖 Machine Learning Utilities
# ==========================================
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.metrics import roc_auc_score

## Data Loading

Mengambil data saham LQ45 dari laman Yahoo Finance

In [4]:
tickers = [
    "ACES.JK", "ADMR.JK", "ADRO.JK", "AKRA.JK", "AMMN.JK", "AMRT.JK", "ANTM.JK", "ARTO.JK",
    "ASII.JK", "BBCA.JK", "BBNI.JK", "BBRI.JK", "BBTN.JK", "BMRI.JK", "BRIS.JK", "BRPT.JK",
    "CPIN.JK", "CTRA.JK", "ESSA.JK", "EXCL.JK", "GOTO.JK", "ICBP.JK", "INCO.JK", "INDF.JK",
    "INKP.JK", "ISAT.JK", "ITMG.JK", "JPFA.JK", "JSMR.JK", "KLBF.JK", "MAPA.JK", "MAPI.JK",
    "MBMA.JK", "MDKA.JK", "MEDC.JK", "PGAS.JK", "PGEO.JK", "PTBA.JK", "SIDO.JK", "SMGR.JK",
    "SMRA.JK", "TLKM.JK", "TOWR.JK", "UNTR.JK", "UNVR.JK"
]

data = yf.download(tickers, start="2025-01-01", end="2025-05-31")['Close']

data_panel = data.stack().reset_index()
data_panel.columns = ['Date', 'Ticker', 'Close']

df_saham = data_panel.copy()
df_saham['Date'] = pd.to_datetime(df_saham['Date'])

df_saham

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  45 of 45 completed


,Date,Ticker,Close
0,2025-01-02,ACES.JK,755.000000
1,2025-01-02,ADMR.JK,1165.000000
2,2025-01-02,ADRO.JK,2530.000000
3,2025-01-02,AKRA.JK,1110.232544
4,2025-01-02,AMMN.JK,8475.000000
...,...,...,...
4090,2025-05-28,SMRA.JK,404.000000
4091,2025-05-28,TLKM.JK,2820.000000
4092,2025-05-28,TOWR.JK,585.000000
4093,2025-05-28,UNTR.JK,22450.000000


Mengambil data BI-rate dari laman resmi BI

In [5]:
df_birate = {
    'Date' : ['2024-12-18', '2025-01-15', '2025-02-19', '2025-03-19', '2025-04-23', '2025-05-21'],
    'BI-Rate' : [0.0600, 0.0575, 0.0575, 0.0575, 0.0575, 0.0550]
}

df_birate = pd.DataFrame(df_birate)
df_birate['Date'] = pd.to_datetime(df_birate['Date'])
df_birate['Start Date'] = df_birate['Date']
df_birate['End Date'] = df_birate['Date'].shift(-1)

last_end_date = df_saham['Date'].max()
df_birate.loc[df_birate.index[-1], 'End Date'] = last_end_date

df_birate

,Date,BI-Rate,Start Date,End Date
0,2024-12-18,0.0600,2024-12-18,2025-01-15
1,2025-01-15,0.0575,2025-01-15,2025-02-19
2,2025-02-19,0.0575,2025-02-19,2025-03-19
3,2025-03-19,0.0575,2025-03-19,2025-04-23
4,2025-04-23,0.0575,2025-04-23,2025-05-21
5,2025-05-21,0.0550,2025-05-21,2025-05-28


In [6]:
wide_df = df_saham.reset_index().pivot(index='Date', columns='Ticker', values='Close').reset_index()

def assign_value(date):
    for i in range(len(df_birate)):
        if df_birate.loc[i, 'Start Date'] <= date <= df_birate.loc[i, 'End Date']:
            return df_birate.loc[i, 'BI-Rate']
    return None

wide_df['BI-Rate'] = wide_df['Date'].apply(assign_value)
df_saham = wide_df.melt(id_vars=['Date', 'BI-Rate'], var_name='Ticker', value_name='Close')

df_saham

,Date,BI-Rate,Ticker,Close
0,2025-01-02,0.060,ACES.JK,755.0
1,2025-01-03,0.060,ACES.JK,755.0
2,2025-01-06,0.060,ACES.JK,755.0
3,2025-01-07,0.060,ACES.JK,735.0
4,2025-01-08,0.060,ACES.JK,705.0
...,...,...,...,...
4090,2025-05-22,0.055,UNVR.JK,1815.0
4091,2025-05-23,0.055,UNVR.JK,1785.0
4092,2025-05-26,0.055,UNVR.JK,1765.0
4093,2025-05-27,0.055,UNVR.JK,1740.0


Mengambil data tingkat inflasi dari laman resmi BI

In [7]:
df_inflasi = {
    'Date' : ['2025-01-01', '2025-02-01', '2025-03-01', '2025-04-01', '2025-05-01'],
    'Rate Inflasi' : [0.0160, 0.0195, 0.0103, -0.0009, 0.0076]
}

df_inflasi = pd.DataFrame(df_inflasi)
df_inflasi['Date'] = pd.to_datetime(df_inflasi['Date'])

df_inflasi

,Date,Rate Inflasi
0,2025-01-01,0.0160
1,2025-02-01,0.0195
2,2025-03-01,0.0103
3,2025-04-01,-0.0009
4,2025-05-01,0.0076


In [8]:
df_saham['Month'] = df_saham['Date'].dt.month
df_inflasi['Month'] = df_inflasi['Date'].dt.month
df_saham = df_saham.merge(df_inflasi[['Month', 'Rate Inflasi']], on='Month', how='left')
df_saham.drop(columns=['Month'], inplace=True)

Mengambil data rate nilai tukar dollar dari laman Yahoo Finance

In [9]:
kurs = yf.download(
    "USDIDR=X",
    start=wide_df['Date'].min().strftime('%Y-%m-%d'),
    end=(wide_df['Date'].max() + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
)

df_kurs = kurs[['Close']].reset_index()
df_kurs.columns = ['Date', 'USD_IDR']

df_kurs

[*********************100%***********************]  1 of 1 completed


,Date,USD_IDR
0,2025-01-02,16230.500000
1,2025-01-03,16201.099609
2,2025-01-06,16178.799805
3,2025-01-07,16179.099609
4,2025-01-08,16157.700195
...,...,...
98,2025-05-22,16347.599609
99,2025-05-23,16356.900391
100,2025-05-26,16211.000000
101,2025-05-27,16243.500000


In [10]:
df_saham = df_saham.merge(df_kurs, left_on='Date', right_on='Date', how='left')
df_saham

,Date,BI-Rate,Ticker,Close,Rate Inflasi,USD_IDR
0,2025-01-02,0.060,ACES.JK,755.0,0.0160,16230.500000
1,2025-01-03,0.060,ACES.JK,755.0,0.0160,16201.099609
2,2025-01-06,0.060,ACES.JK,755.0,0.0160,16178.799805
3,2025-01-07,0.060,ACES.JK,735.0,0.0160,16179.099609
4,2025-01-08,0.060,ACES.JK,705.0,0.0160,16157.700195
...,...,...,...,...,...,...
4090,2025-05-22,0.055,UNVR.JK,1815.0,0.0076,16347.599609
4091,2025-05-23,0.055,UNVR.JK,1785.0,0.0076,16356.900391
4092,2025-05-26,0.055,UNVR.JK,1765.0,0.0076,16211.000000
4093,2025-05-27,0.055,UNVR.JK,1740.0,0.0076,16243.500000


In [11]:
df_saham['Trump Tariffs'] = (df_saham.Date >= pd.to_datetime('2025-04-05')).astype(int)
df_saham

,Date,BI-Rate,Ticker,Close,Rate Inflasi,USD_IDR,Trump Tariffs
0,2025-01-02,0.060,ACES.JK,755.0,0.0160,16230.500000,0
1,2025-01-03,0.060,ACES.JK,755.0,0.0160,16201.099609,0
2,2025-01-06,0.060,ACES.JK,755.0,0.0160,16178.799805,0
3,2025-01-07,0.060,ACES.JK,735.0,0.0160,16179.099609,0
4,2025-01-08,0.060,ACES.JK,705.0,0.0160,16157.700195,0
...,...,...,...,...,...,...,...
4090,2025-05-22,0.055,UNVR.JK,1815.0,0.0076,16347.599609,1
4091,2025-05-23,0.055,UNVR.JK,1785.0,0.0076,16356.900391,1
4092,2025-05-26,0.055,UNVR.JK,1765.0,0.0076,16211.000000,1
4093,2025-05-27,0.055,UNVR.JK,1740.0,0.0076,16243.500000,1


In [12]:
df_saham.sort_values(by=['Ticker', 'Date'], inplace=True)
df_saham.set_index(['Ticker', 'Date'], inplace=True)
df_saham

BI-Rate   Close  Rate Inflasi       USD_IDR  Trump Tariffs
Ticker  Date                                                                  
ACES.JK 2025-01-02    0.060   755.0        0.0160  16230.500000              0
        2025-01-03    0.060   755.0        0.0160  16201.099609              0
        2025-01-06    0.060   755.0        0.0160  16178.799805              0
        2025-01-07    0.060   735.0        0.0160  16179.099609              0
        2025-01-08    0.060   705.0        0.0160  16157.700195              0
...                     ...     ...           ...           ...            ...
UNVR.JK 2025-05-22    0.055  1815.0        0.0076  16347.599609              1
        2025-05-23    0.055  1785.0        0.0076  16356.900391              1
        2025-05-26    0.055  1765.0        0.0076  16211.000000              1
        2025-05-27    0.055  1740.0        0.0076  16243.500000              1
        2025-05-28    0.055  1730.0        0.0076  16279.000000              1

[4095 rows x 5 columns]

## Model Development

Data Lengkap setelah disatukan

In [13]:
df_saham['Return'] = df_saham.groupby('Ticker')['Close'].pct_change()

def label_return(x):
    if x < -0.01:
        return 'Turun'
    elif x > 0.01:
        return 'Naik'
    else:
        return 'Stagnan'

df_saham['Kategori'] = df_saham['Return'].apply(label_return)

In [14]:
df_saham.dropna(inplace=True)
df_saham

BI-Rate   Close  Rate Inflasi       USD_IDR  \
Ticker  Date                                                      
ACES.JK 2025-01-03    0.060   755.0        0.0160  16201.099609   
        2025-01-06    0.060   755.0        0.0160  16178.799805   
        2025-01-07    0.060   735.0        0.0160  16179.099609   
        2025-01-08    0.060   705.0        0.0160  16157.700195   
        2025-01-09    0.060   720.0        0.0160  16187.599609   
...                     ...     ...           ...           ...   
UNVR.JK 2025-05-22    0.055  1815.0        0.0076  16347.599609   
        2025-05-23    0.055  1785.0        0.0076  16356.900391   
        2025-05-26    0.055  1765.0        0.0076  16211.000000   
        2025-05-27    0.055  1740.0        0.0076  16243.500000   
        2025-05-28    0.055  1730.0        0.0076  16279.000000   

                    Trump Tariffs    Return Kategori  
Ticker  Date                                          
ACES.JK 2025-01-03              0  0.000000  Stagnan  
        2025-01-06              0  0.000000  Stagnan  
        2025-01-07              0 -0.026490    Turun  
        2025-01-08              0 -0.040816    Turun  
        2025-01-09              0  0.021277     Naik  
...                           ...       ...      ...  
UNVR.JK 2025-05-22              1  0.031250     Naik  
        2025-05-23              1 -0.016529    Turun  
        2025-05-26              1 -0.011204    Turun  
        2025-05-27              1 -0.014164    Turun  
        2025-05-28              1 -0.005747  Stagnan  

[4005 rows x 7 columns]

mengubah variabel dependen ke bentuk numerik

In [15]:
df_saham['Close_lag1'] = df_saham.groupby('Ticker')['Close'].shift(1)
df_saham['BI-Rate_lag1'] = df_saham.groupby('Ticker')['BI-Rate'].shift(1)
df_saham['Inflasi_lag1'] = df_saham.groupby('Ticker')['Rate Inflasi'].shift(1)
df_saham['USD_IDR_lag1'] = df_saham.groupby('Ticker')['USD_IDR'].shift(1)
df_saham = df_saham.dropna()

le = LabelEncoder()
df_saham['Target_encoded'] = le.fit_transform(df_saham['Kategori'])

<ipython-input-15-36aa34181b3e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_saham['Target_encoded'] = le.fit_transform(df_saham['Kategori'])


Ringakasan dari data yang digunakan

In [16]:
df_saham.describe()

,BI-Rate,Close,Rate Inflasi,USD_IDR,Trump Tariffs,Return,Close_lag1,BI-Rate_lag1,Inflasi_lag1,USD_IDR_lag1,Target_encoded
count,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000
mean,0.057585,3524.312775,0.011061,16435.539739,0.363636,0.000282,3526.282773,0.057642,0.011157,16434.654508,1.028535
std,0.000957,4958.220143,0.006909,208.121415,0.481106,0.033187,4965.966878,0.000950,0.006919,208.949599,0.800634
min,0.055000,64.000000,-0.000900,16157.700195,0.000000,-0.156951,66.000000,0.055000,-0.000900,16157.700195,0.000000
25%,0.057500,945.000000,0.007600,16289.274658,0.000000,-0.016534,945.000000,0.057500,0.007600,16289.274658,0.000000
50%,0.057500,1893.897644,0.010300,16358.850098,0.000000,0.000000,1891.397644,0.057500,0.010300,16358.850098,1.000000
75%,0.057500,3995.890015,0.016000,16542.150391,1.000000,0.014426,3995.890015,0.057500,0.016000,16542.150391,2.000000
max,0.060000,26425.000000,0.019500,17051.900391,1.000000,0.222222,26425.000000,0.060000,0.019500,17051.900391,2.000000


Cek multikolinearitas dari variabel


In [28]:
X = df_saham.drop(columns=['Close', 'BI-Rate', 'Rate Inflasi', 'USD_IDR', 'Return', 'Kategori', 'Target_encoded'])
X = sm.add_constant(X)

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data.drop(index=0)

,Variable,VIF
1,Trump Tariffs,3.248884
2,Close_lag1,1.000957
3,BI-Rate_lag1,1.162335
4,Inflasi_lag1,5.029743
5,USD_IDR_lag1,3.298968


Seluruh nilai VIF berada di bawah ambang batas umum sebesar $10$, sehingga **tidak terdapat indikasi multikolinearitas tinggi** dalam model. Nilai VIF tertinggi terdapat pada `Inflasi_lag1` ($5.03$) yang masih dalam batas wajar namun perlu diperhatikan karena mendekati nilai moderat. Variabel `Trump Tariffs` dan `USD_IDR_lag1` menunjukkan korelasi sedang, sementara `Close_lag1` dan `BI-Rate_lag1` menunjukkan korelasi sangat rendah. Dengan demikian, seluruh variabel masih layak untuk dipertahankan dalam model.

In [29]:
y = df_saham['Target_encoded']

model = sm.MNLogit(y, X)
result = model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 1.070669
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:         Target_encoded   No. Observations:                 3960
Model:                        MNLogit   Df Residuals:                     3948
Method:                           MLE   Df Model:                           10
Date:                Sat, 07 Jun 2025   Pseudo R-squ.:                 0.02364
Time:                        17:54:00   Log-Likelihood:                -4239.8
converged:                       True   LL-Null:                       -4342.5
Covariance Type:            nonrobust   LLR p-value:                 1.278e-38
Target_encoded=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               47.5264      6.417      7.406      0.000      34.949      60.104
Trump Tarif

Model multinomial logit berhasil dikonvergensi dan menghasilkan pseudo R-squared sebesar $0,0236$, yang menunjukkan bahwa model menjelaskan sekitar $2,4\%$ variasi pada variabel target. Meskipun nilai ini tergolong rendah, hal tersebut cukup umum dalam model logit, terutama untuk data panel dengan kompleksitas tinggi dan banyak noise.

Untuk kategori `Target = 1` (dibandingkan dengan referensi `Target = 0`), variabel `Close_lag1`, `Inflasi_lag1`, dan `USD_IDR_lag1` berpengaruh signifikan. `Close_lag1` memiliki koefisien positif, artinya kenaikan harga penutupan sebelumnya meningkatkan kemungkinan masuk ke kategori ini. Sebaliknya, `Inflasi_lag1` dan `USD_IDR_lag1` berkoefisien negatif dan signifikan, menunjukkan bahwa inflasi dan nilai tukar USD/IDR yang lebih tinggi cenderung menurunkan peluang berada di kategori ini.

Untuk kategori `Target = 2`, hanya `USD_IDR_lag1` yang signifikan dan berpengaruh negatif. Variabel `Trump Tariffs` tidak signifikan dalam kedua kategori, sehingga belum terdapat bukti kuat bahwa kebijakan tarif Trump berdampak langsung terhadap perubahan klasifikasi target pada periode pengamatan ini.

Secara keseluruhan, model menunjukkan adanya pengaruh signifikan dari beberapa variabel ekonomi terhadap perubahan kategori target, namun kontribusi model terhadap variasi target masih terbatas.


## Kesimpulan

Penelitian ini menggunakan model multinomial logit dengan pendekatan data panel untuk menganalisis pengaruh kebijakan **Trump Tariffs** dan variabel makroekonomi terhadap kategori perubahan harga saham harian perusahaan dalam indeks LQ45 Bursa Efek Indonesia.

Hasil estimasi menunjukkan bahwa variabel seperti nilai tukar USD/IDR dan inflasi memiliki pengaruh signifikan terhadap probabilitas perubahan harga saham, sedangkan efek langsung dari penerapan tarif impor Amerika Serikat (**Trump Tariffs**) terhadap kategori perubahan harga saham tidak signifikan secara statistik dalam periode pengamatan ini.

Penggunaan model multinomial logit memungkinkan pemahaman yang lebih kaya mengenai probabilitas relatif saham berada dalam kategori naik, turun, atau stabil, dibandingkan dengan model regresi linier biasa yang hanya memodelkan harga secara kontinu.

Penelitian ini menegaskan pentingnya mempertimbangkan faktor makroekonomi dan dinamika harga saham sebelumnya (lag) dalam memodelkan pergerakan saham. Untuk penelitian selanjutnya, dapat dipertimbangkan penggunaan model dengan efek acak atau metode lain yang dapat menangani ketergantungan temporal dan heterogenitas lebih kompleks.

Secara praktis, hasil ini memberikan gambaran bahwa pasar saham Indonesia dalam indeks LQ45 lebih responsif terhadap variabel ekonomi domestik dan nilai tukar daripada perubahan kebijakan tarif impor Amerika Serikat secara langsung dalam jangka pendek.


# Sumber

[[1](https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration)] "Tariffs in the second Trump administration", Wikipedia, [Online]. Tersedia: https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration. [Diakses: Jun. 5, 2025]. \
[[2](https://id.wikipedia.org/wiki/Indeks_LQ45)] "Indeks LQ45", Wikipedia, [Online]. Tersedia: https://id.wikipedia.org/wiki/Indeks_LQ45. [Diakses: Jun. 5, 2025]. \
[3] E. W. Frees, Longitudinal and Panel Data: Analysis and Applications in the Social Sciences. Cambridge: Cambridge University Press, 2004.